In [ ]:
import re # 정규 표현식을 사용하기 위해 import (더 유연한 방법)

# --- 설정 ---
input_file_path = 'ETM_Results.txt'  # <<<< 원본 파일 경로를 여기에 입력하세요
output_file_path = 'ETM_Output.txt' # <<<< 저장될 파일 경로를 여기에 입력하세요

# 제거할 특수 문자 목록 (문자열로 정의)
# 주의: 정규 표현식에서 특별한 의미를 가지는 문자는 앞에 \를 붙여 이스케이프해야 할 수 있습니다.
# 여기서는 [ 가 특수 의미를 가지므로 \를 붙입니다. ' 와 , 는 그대로 사용해도 괜찮습니다.
# 만약 단순히 문자 리스트로 처리하고 싶다면 아래 '방법 2'를 보세요.
chars_to_remove_pattern = r"[\[',\']" # 정규표현식 패턴: '[' 또는 ',' 또는 "'" 를 의미

# 파일 인코딩 설정 (파일이 UTF-8이 아닌 경우 'cp949' 등으로 변경)
file_encoding = 'utf-8'
# --- 설정 끝 ---

print(f"입력 파일: {input_file_path}")
print(f"출력 파일: {output_file_path}")
print(f"제거할 문자 패턴: {chars_to_remove_pattern}")

try:
    with open(input_file_path, 'r', encoding=file_encoding) as infile:
        original_content = infile.read()
    print(f"'{input_file_path}' 파일 읽기 완료.")

    cleaned_content = re.sub(chars_to_remove_pattern, '', original_content)

    with open(output_file_path, 'w', encoding=file_encoding) as outfile:
        outfile.write(cleaned_content)
    print(f"처리된 내용을 '{output_file_path}' 파일에 저장 완료.")

    print("작업 완료!")

except FileNotFoundError:
    print(f"오류: 입력 파일 '{input_file_path}'을(를) 찾을 수 없습니다. 파일 경로를 확인해주세요.")
except Exception as e:
    print(f"오류 발생: {e}")

입력 파일: ETM_Results.txt
출력 파일: ETM_Output.txt
제거할 문자 패턴: [\[',\']
'ETM_Results.txt' 파일 읽기 완료.
처리된 내용을 'ETM_Output.txt' 파일에 저장 완료.
작업 완료!


In [ ]:
# 최종

import pandas as pd
import torch
from transformers import pipeline # pipeline을 직접 사용
from tqdm.auto import tqdm

# --- 1. 설정 ---
CSV_FILE_PATH = 'emotion.csv'
KEYWORD_COLUMN_NAME = 'topic'
MODEL_NAME = "bhadresh-savani/distilbert-base-uncased-emotion"
# 결과 파일 이름에 모델 이름을 포함시켜 구분 용이하게 함
OUTPUT_CSV_FILE_PATH = f'topics_with_emotions_{MODEL_NAME.split("/")[-1]}.csv'

# --- 2. 모델 로드 ---
print(f"Loading text classification pipeline for model: {MODEL_NAME}...")
try:
    # top_k=None : 모든 감정 레이블과 점수를 반환하도록 설정
    emotion_classifier = pipeline("text-classification",
                                  model=MODEL_NAME,
                                  top_k=None) # 모든 클래스 점수 받기
    print("Pipeline loaded successfully.")

    # 모델이 사용하는 감정 레이블 확인
    try:
        emotion_labels = list(emotion_classifier.model.config.id2label.values())
        print(f"Detected emotion labels: {emotion_labels}")
    except AttributeError:
        print("Could not automatically detect emotion labels from model config.")
        emotion_labels = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'] # 모델 기본 레이블로 가정
        print(f"Assuming default labels: {emotion_labels}")

except Exception as e:
    print(f"Error loading pipeline: {e}")
    exit()

# GPU 사용 가능하면 pipeline이 자동으로 사용 시도함
print(f"Pipeline using device: {emotion_classifier.device}")

# --- 3. CSV 파일 로드 ---
print(f"Loading CSV file: {CSV_FILE_PATH}...")
try:
    df = pd.read_csv(CSV_FILE_PATH)
    if KEYWORD_COLUMN_NAME not in df.columns:
        raise ValueError(f"Error: Column '{KEYWORD_COLUMN_NAME}' not found in the CSV file.")
    print(f"CSV file loaded. Found {len(df)} topics.")
except FileNotFoundError:
    print(f"Error: CSV file not found at {CSV_FILE_PATH}")
    exit()
except Exception as e:
    print(f"Error reading CSV file: {e}")
    exit()

# --- 4. 감정 분석 함수 정의 (pipeline 사용 버전) ---
def analyze_topic_emotions_distilbert_emotion(keywords_str):
    if not isinstance(keywords_str, str) or not keywords_str.strip():
        # 반환 구조: 모든_점수_딕셔너리, 가장_높은_감정, 가장_높은_점수, 사용된_입력_텍스트
        return {}, None, None, "Input keywords are empty or invalid."

    # --- 입력 템플릿 사용 ---
    input_text = f"Explore the emotional landscape surrounding a villain character whose motivations include: {keywords_str}"

    try:
        results = emotion_classifier(input_text)

        # --- 결과 처리 ---
        if results and isinstance(results[0], list):
            all_scores_list = results[0]
            all_scores_dict = {item['label']: item['score'] for item in all_scores_list}

            # 가장 높은 점수를 가진 감정 찾기
            dominant_emotion = None
            dominant_score = None
            if all_scores_list:
                 # 첫 번째 항목이 가장 높은 점수
                dominant_emotion = all_scores_list[0]['label']
                dominant_score = all_scores_list[0]['score']

            return all_scores_dict, dominant_emotion, dominant_score, input_text
        else:
            print(f"Warning: Unexpected output format from pipeline for input: '{input_text[:100]}...'. Output: {results}")
            return {}, None, None, input_text

    except Exception as e:
        print(f"Error during analysis for input: '{input_text[:100]}...'. Error: {e}")
        return {}, None, None, f"Error during analysis: {e}"


# --- 5. 각 토픽에 대해 감정 분석 수행 ---
analysis_results = [] # 결과 리스트
print(f"\nStarting emotion analysis using {MODEL_NAME}...")

for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Analyzing Topics"):
    keywords = row[KEYWORD_COLUMN_NAME]

    # 분석 함수 호출
    all_emotions, dominant_emotion, dominant_score, processed_text = analyze_topic_emotions_distilbert_emotion(keywords)

    result_row = {
        'topic_id': row.get('topic_id', index),
        'keywords': keywords,
        'processed_text_for_model': processed_text,
        'all_emotion_scores': all_emotions,   # 모든 감정 점수 딕셔너리
        'dominant_emotion': dominant_emotion, # 가장 높은 점수의 감정
        'dominant_score': dominant_score    # 해당 점수
    }
    analysis_results.append(result_row)

print("Emotion analysis complete.")

# --- 6. 결과를 DataFrame으로 변환 및 저장 ---
results_df = pd.DataFrame(analysis_results)

print("\n--- Analysis Results (First 5 rows) ---")
# 주요 결과 열 위주로 출력 (딕셔너리가 길 수 있으므로)
print(results_df[['topic_id', 'keywords', 'dominant_emotion', 'dominant_score']].head())

print(f"\nSaving results to {OUTPUT_CSV_FILE_PATH}...")
try:
    results_df.to_csv(OUTPUT_CSV_FILE_PATH, index=False, encoding='utf-8-sig')
    print("Results saved successfully.")
except Exception as e:
    print(f"Error saving results to CSV: {e}")

Loading text classification pipeline for model: bhadresh-savani/distilbert-base-uncased-emotion...


Device set to use cpu


Pipeline loaded successfully.
Detected emotion labels: ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
Pipeline using device: cpu
Loading CSV file: emotion.csv...
CSV file loaded. Found 299 topics.

Starting emotion analysis using bhadresh-savani/distilbert-base-uncased-emotion...


Analyzing Topics:   0%|          | 0/299 [00:00<?, ?it/s]

Emotion analysis complete.

--- Analysis Results (First 5 rows) ---
   topic_id  keywords dominant_emotion  dominant_score
0         0    murder             fear        0.525356
1         1   commits          sadness        0.708104
2         2      kind          sadness        0.594660
3         3    family          sadness        0.623607
4         4  personal          sadness        0.641220

Saving results to topics_with_emotions_distilbert-base-uncased-emotion.csv...
Results saved successfully.


In [ ]:
import pandas as pd

# --- 1. 설정 ---
FINAL_RESULT_CSV_PATH = 'final_result.csv'

DOMINANT_EMOTION_COLUMN = 'dominant_emotion'

OUTPUT_SADNESS_CSV_PATH = 'topics_only_sadness.csv'

# --- 2. 최종 결과 CSV 파일 로드 ---
print(f"Loading final result CSV: {FINAL_RESULT_CSV_PATH}")
try:
    df_final = pd.read_csv(FINAL_RESULT_CSV_PATH)

    if DOMINANT_EMOTION_COLUMN not in df_final.columns:
        raise ValueError(f"Error: Column '{DOMINANT_EMOTION_COLUMN}' not found in the CSV file.")

    print(f"Loaded {len(df_final)} total topics from the file.")

except FileNotFoundError:
    print(f"Error: CSV file not found at {FINAL_RESULT_CSV_PATH}")
    exit()
except Exception as e:
    print(f"Error reading CSV file: {e}")
    exit()

# --- 3. Dominant Emotion 종류별 개수 확인 및 출력 ---
print(f"\nCalculating counts for each unique value in the '{DOMINANT_EMOTION_COLUMN}' column...")

try:
    # .value_counts() 메서드를 사용하여 각 감정별 빈도수 계산
    emotion_counts = df_final[DOMINANT_EMOTION_COLUMN].value_counts()

    print("\n--- Dominant Emotion Counts ---")
    if not emotion_counts.empty:
        print(emotion_counts)
    else:
        print(f"The '{DOMINANT_EMOTION_COLUMN}' column might be empty or contain only NaN values.")

# 에러 확인
except KeyError:
    print(f"Error: Column '{DOMINANT_EMOTION_COLUMN}' seems to be missing after loading.")
    exit()
except Exception as e:
    print(f"An error occurred while calculating value counts: {e}")

# --- 4. 'dominant_emotion' 열에서 'sadness'인 행만 필터링 ---
target_emotion = 'sadness'
print(f"\nFiltering topics where '{DOMINANT_EMOTION_COLUMN}' is '{target_emotion}'...")

try:
    # boolean indexing을 사용하여 'sadness' 행만 선택
    df_sadness_only = df_final[df_final[DOMINANT_EMOTION_COLUMN] == target_emotion].copy()

    print(f"Found {len(df_sadness_only)} topics where the dominant emotion is '{target_emotion}'.")

except KeyError:
     print(f"Error: Cannot filter because column '{DOMINANT_EMOTION_COLUMN}' is missing.")
     df_sadness_only = pd.DataFrame()
except Exception as e:
     print(f"An error occurred during filtering: {e}")
     df_sadness_only = pd.DataFrame()

# --- 5. 필터링된 결과 확인 ---
if not df_sadness_only.empty:
    print("\n--- Filtered Results ('sadness' only - First 5 rows) ---")
    print(df_sadness_only.head())
else:
    print(f"\nNo topics were found with '{target_emotion}' as the dominant emotion, or an error occurred during filtering.")

# --- 6. 필터링된 DataFrame을 새 CSV 파일로 저장 ---
if not df_sadness_only.empty:
    print(f"\nSaving the filtered '{target_emotion}' topics to {OUTPUT_SADNESS_CSV_PATH}...")
    try:
        df_sadness_only.to_csv(OUTPUT_SADNESS_CSV_PATH, index=False, encoding='utf-8-sig')
        print("Filtered results saved successfully.")
    except Exception as e:
        print(f"Error saving filtered results to CSV: {e}")
else:
    print(f"\nSkipping save because no '{target_emotion}' topics were found or an error occurred.")

print("\nScript finished.")

Loading final result CSV: final_result.csv
Loaded 299 total topics from the file.

Calculating counts for each unique value in the 'dominant_emotion' column...

--- Dominant Emotion Counts ---
dominant_emotion
sadness    188
fear        53
anger       29
joy         25
love         4
Name: count, dtype: int64

Filtering topics where 'dominant_emotion' is 'sadness'...
Found 188 topics where the dominant emotion is 'sadness'.

--- Filtered Results ('sadness' only - First 5 rows) ---
   topic_id  keywords                           processed_text_for_model  \
1         1   commits  Explore the emotional landscape surrounding a ...   
2         2      kind  Explore the emotional landscape surrounding a ...   
3         3    family  Explore the emotional landscape surrounding a ...   
4         4  personal  Explore the emotional landscape surrounding a ...   
5         5    enjoys  Explore the emotional landscape surrounding a ...   

                                  all_emotion_scores domi

**1. Extraversion(외향성)**

*   사교성과 활동성이라는 두 가지 요소로 구성
*   따뜻하고, 긍정적인 감정을 자주 느끼며, 에너지가 넘치고, 자기 주장이 강하며, 외향적인 성향을 지닌 것


**2.   Agreeableness(우호성)**
*   개인이 대인관계를 어떻게 접근하는지를 나타내는 특성입니다. 이 특성이 높은 사람들은 타인을 배려하고, 협력적이며, 갈등을 피하려는 경향


**3. Neuroticism(신경증)**

*   불안, 죄책감, 불안정함, 자기 연민 같은 부정적인 감정을 자주 경험하는 경향이 있으며, 감정 기복이 심하고 충동적인 행동


**4. Conscientiousness(성실성)**

*   계획적이고 조직적인 태도를 가지며, 목표 지향적이고 즉각적인 만족을 미루며, 사회적 규범과 규칙을 잘 따르는 성향


**5. Openness(개방성)**

*   지적 호기심이 많고 상상력이 풍부한 특징




# 개방성
['']

In [ ]:
import pandas as pd
import ast
from tqdm.auto import tqdm

# --- 1. 설정 ---
INPUT_CSV_PATH = 'topics_only_sadness.csv'
OUTPUT_CSV_PATH = 'topics_sadness_mapped_to_big5_user_keywords.csv'
KEYWORDS_COLUMN = 'keywords'
DOMINANT_EMOTION_COLUMN = 'dominant_emotion'

# --- 2. Big 5 연관 키워드 셋 정의  ---

# 높은 개방성: 새로운 걸 추구하고, 변화하거나 호기심을 가짐
KEYWORDS_HIGH_OPENNESS = ['change', 'knowledge', 'complex', 'wizard', 'archaeologist', 'unresolved', 'issue', 'twist'] # archaeolgist 오타 수정
# 낮은 개방성: 보수적, 전통에 머무름, 변화 추구 X
KEYWORDS_LOW_OPENNESS = ['belief', 'uniform', 'authority', 'perfectly', 'nazi', 'german']

# 계획성, 완벽주의, (스스로 + 상대를) 통제
KEYWORDS_HIGH_CONSCIENTIOUSNESS = ['planning', 'planner', 'meticulously', 'execute', 'perfectly', 'control', 'fulfill', 'determination', 'work', 'goal', 'leader', 'commander', 'boss', 'tycoon',  'politician', 'order', 'solve']
KEYWORDS_LOW_CONSCIENTIOUSNESS  = ['disorder', 'destruction', 'destroy', 'unpredictable', 'outburst', 'instability', 'consume', 'drug', 'extravagance', 'careless', 'impulsive']

# 사교성, 지배성, 활동성
KEYWORDS_HIGH_EXTRAVERSION = ['fluent', 'show', 'laughter', 'joke', 'humor', 'amuse', 'actor', 'power', 'mass', 'leader', 'commander', 'boss', 'tycoon', 'politician']
# 내향성, 조용함, 수동적
KEYWORDS_LOW_EXTRAVERSION = ['loneliness', 'seclusion', 'secrets', 'observation', 'quiet', 'withdrawn', 'expressionless', 'alone']

# 위선, 이중성 키워드
KEYWORDS_FEIGNED_AGREEABLENESS  = ['humanity', 'kind', 'two-face', 'extravagance', 'switch', 'good', 'nature', 'manner', 'pretend', 'victim']
# 비협조적, 적대성
KEYWORDS_LOW_AGREEABLENESS = ['manipulation', 'control', 'betrayal', 'deception', 'mean', 'cruelty', 'ruthlessness', 'abuse', 'mistreatment', 'torment', 'inflicting', 'pain', 'selfishness', 'gain', 'dislike', 'hate', 'misogyny', 'racism', 'discrimination', 'slaves', 'revenge', 'unforgiving', 'hostile', 'suspicious', 'scar', 'enrage', 'shape', 'remorse', 'commit', 'criminal', 'crime', 'attempt', 'genocide', 'massacre', 'empathy'] # torment 오타 수정, empathy(부족) 이전 답변에서 추가함 (유지)

# 불안, 분노
KEYWORDS_HIGH_NEUROTICISM  = ['anger', 'depression', 'pessimism', 'fear', 'paranoia', 'past', 'trauma', 'stress', 'madness', 'delusion', 'instability', 'complex', 'obsession', 'solely', 'specific', 'goal', 'lack', 'vulnerable', 'cursed', 'collapse', 'teardrop', 'guilty', 'feel', 'pain'] # sadness 자체는 여기서 제외 (대표 감정으로 처리), teardrop, guilty, feel, pain 이전 답변에서 추가함 (유지)

KEYWORDS_LOW_NEUROTICISM = ['calm', 'stable', 'confident', 'resilient']


# --- 3. Sadness 해석 함수 (Big 5 관점) ---
def interpret_sadness_through_big5(keywords):
    interpreted_flavor = 'General Sadness'
    profile_adjustments = {'Neuroticism': 'High'} # 기본적으로 슬픔은 신경성이 높음

    # 규칙 1: Feigned/Manipulative Sadness (가짜 슬픔) -> 낮은 우호성
    if any(k in keywords for k in KEYWORDS_FEIGNED_AGREEABLENESS):
        interpreted_flavor = 'Feigned Sadness / Manipulative Pity Play'
        profile_adjustments = {'Agreeableness': 'Very Low', 'Conscientiousness': 'High', 'Neuroticism': 'Medium'}
        return interpreted_flavor, profile_adjustments

    # 규칙 2: Vengeful Sadness (복수심/원한) -> 매우 높은 신경성, 매우 낮은 우호성
    revenge_keywords = ['revenge', 'betrayal', 'unforgiving', 'hostile', 'hate', 'anger']
    if any(k in keywords for k in revenge_keywords):
        interpreted_flavor = 'Vengeful Sadness / Bitter Resentment'
        profile_adjustments = {'Neuroticism': 'Very High', 'Agreeableness': 'Very Low'}
        return interpreted_flavor, profile_adjustments

    # 규칙 3: Lonely Despair (고립/절망) -> 높은 신경성, 매우 낮은 외향성
    isolation_keywords = ['loneliness', 'seclusion', 'alone', 'empty', 'despair', 'loss']
    if any(k in keywords for k in isolation_keywords):
        interpreted_flavor = 'Lonely Despair / Isolated Sorrow'
        profile_adjustments = {'Neuroticism': 'High', 'Extraversion': 'Very Low'}
        return interpreted_flavor, profile_adjustments

    # 규칙 4: Tragic Resignation (비극/숙명) -> 높은 신경성, 낮은 개방성?
    tragic_keywords = ['sacrifice', 'duty', 'burden', 'fate', 'destiny', 'resignation', 'cursed', 'tragic']
    if any(k in keywords for k in tragic_keywords):
        interpreted_flavor = 'Tragic Resignation / World-Weary Sadness'
        profile_adjustments = {'Neuroticism': 'High', 'Openness': 'Low'}
        return interpreted_flavor, profile_adjustments

    # 규칙 5: Haunting Sorrow (과거 트라우마) -> 높은 신경성
    past_trauma_keywords = ['past', 'trauma', 'memory', 'grief', 'suffering', 'haunted', 'collapse', 'stress']
    if any(k in keywords for k in past_trauma_keywords):
        interpreted_flavor = 'Haunting Sorrow / Past Trauma Sadness'
        profile_adjustments = {'Neuroticism': 'High'}
        return interpreted_flavor, profile_adjustments

    # 아무 규칙에도 해당하지 않으면 기본값 반환
    return interpreted_flavor, profile_adjustments


# --- 4. Big 5 프로파일 매핑 함수 (사용자 키워드 셋 기반) ---
def map_to_big5_with_user_keywords(keywords_str, dominant_emotion):
    """사용자 정의 키워드 셋과 대표 감정을 바탕으로 Big 5 프로파일과 감정 뉘앙스를 해석합니다."""
    profile = { "Openness": "Medium", "Conscientiousness": "Medium", "Extraversion": "Medium",
                "Agreeableness": "Medium", "Neuroticism": "Medium" }
    interpreted_flavor = dominant_emotion if pd.notna(dominant_emotion) else "Unknown"

    if isinstance(keywords_str, str):
        keywords = [k.strip().lower() for k in keywords_str.split(',') if k.strip()] # 소문자 변환
    elif isinstance(keywords_str, list):
        keywords = [str(k).strip().lower() for k in keywords_str if str(k).strip()] # 리스트 내 요소도 문자열 변환 및 소문자화
    else: keywords = []

    # --- 단계 1: 직접적인 Big 5 키워드 영향 적용 ---
    if any(k in keywords for k in KEYWORDS_HIGH_OPENNESS): profile['Openness'] = 'High'
    if any(k in keywords for k in KEYWORDS_LOW_OPENNESS): profile['Openness'] = 'Low'
    if any(k in keywords for k in KEYWORDS_HIGH_CONSCIENTIOUSNESS): profile['Conscientiousness'] = 'High'
    if any(k in keywords for k in KEYWORDS_LOW_CONSCIENTIOUSNESS): profile['Conscientiousness'] = 'Low'
    if any(k in keywords for k in KEYWORDS_HIGH_EXTRAVERSION): profile['Extraversion'] = 'High'
    if any(k in keywords for k in KEYWORDS_LOW_EXTRAVERSION): profile['Extraversion'] = 'Low'

    # Agreeableness 처리: 위선 키워드는 Very Low, 낮은 우호성 키워드는 Low로 설정
    if any(k in keywords for k in KEYWORDS_FEIGNED_AGREEABLENESS):
        profile['Agreeableness'] = 'Very Low'
        interpreted_flavor = 'Feigned Positivity' # 감정 맛에도 영향 줄 수 있음 (sadness 아니더라도)
    elif any(k in keywords for k in KEYWORDS_LOW_AGREEABLENESS):
         # 위선 키워드가 없을 때만 Low 설정 (Very Low가 더 강함)
        if profile['Agreeableness'] != 'Very Low':
             profile['Agreeableness'] = 'Low'

    if any(k in keywords for k in KEYWORDS_HIGH_NEUROTICISM): profile['Neuroticism'] = 'High'
    if any(k in keywords for k in KEYWORDS_LOW_NEUROTICISM): profile['Neuroticism'] = 'Low'

    # --- 단계 2: 대표 감정 해석 및 프로파일 조정 (Sadness 집중) ---
    if dominant_emotion == 'sadness':
        # sadness 해석 함수 호출
        interpreted_flavor, sadness_adjustments = interpret_sadness_through_big5(keywords)

        # Sadness 해석 결과로 프로파일 업데이트
        level_map = {'Very Low': 1, 'Low': 2, 'Medium': 3, 'High': 4, 'Very High': 5}
        reverse_level_map = {v: k for k, v in level_map.items()}

        for trait, adj_level_str in sadness_adjustments.items():
            current_level_str = profile[trait]
            current_level_num = level_map.get(current_level_str, 3)
            adj_level_num = level_map.get(adj_level_str, 3)

            if current_level_num != adj_level_num :
                 profile[trait] = adj_level_str

    elif dominant_emotion and interpreted_flavor == dominant_emotion and any(k in keywords for k in KEYWORDS_FEIGNED_AGREEABLENESS):
         interpreted_flavor = f'Feigned {dominant_emotion}'


    return profile, interpreted_flavor


# --- 5. CSV 파일 로드 및 처리 ---
print(f"Loading input CSV: {INPUT_CSV_PATH}")
try:
    df = pd.read_csv(INPUT_CSV_PATH)
    print(f"Loaded {len(df)} topics.")
except Exception as e:
    print(f"Error loading or processing input CSV: {e}")
    exit()

# --- 6. 각 토픽에 대해 Big 5 매핑 수행 ---
big5_results = []
print("Mapping keywords and emotions to Big 5 profiles using user-defined keywords...")
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Mapping to Big 5"):
    keywords_data = row[KEYWORDS_COLUMN]
    dominant_emotion_data = row[DOMINANT_EMOTION_COLUMN]

    big5_profile, interpreted_emotion_flavor = map_to_big5_with_user_keywords(keywords_data, dominant_emotion_data)

    result_row = row.to_dict()
    result_row['big5_profile_mapped'] = big5_profile
    result_row['interpreted_emotion_flavor'] = interpreted_emotion_flavor
    big5_results.append(result_row)

print("Big 5 mapping complete.")

# --- 7. 결과를 새 DataFrame으로 변환 및 저장 ---
results_df = pd.DataFrame(big5_results)

print("\n--- Mapping Results (First 5 rows) ---")

print(results_df[['topic_id', 'keywords', 'dominant_emotion', 'interpreted_emotion_flavor', 'big5_profile_mapped']].head())

print(f"\nSaving results to {OUTPUT_CSV_PATH}...")
try:
    results_df.to_csv(OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
    print("Results saved successfully.")
except Exception as e:
    print(f"Error saving results to CSV: {e}")

print("\nScript finished.")

Loading input CSV: topics_only_sadness.csv
Loaded 188 topics.
Mapping keywords and emotions to Big 5 profiles using user-defined keywords...


Mapping to Big 5:   0%|          | 0/188 [00:00<?, ?it/s]

Big 5 mapping complete.

--- Mapping Results (First 5 rows) ---
   topic_id  keywords dominant_emotion  \
0         1   commits          sadness   
1         2      kind          sadness   
2         3    family          sadness   
3         4  personal          sadness   
4         5    enjoys          sadness   

                 interpreted_emotion_flavor  \
0                           General Sadness   
1  Feigned Sadness / Manipulative Pity Play   
2                           General Sadness   
3                           General Sadness   
4                           General Sadness   

                                 big5_profile_mapped  
0  {'Openness': 'Medium', 'Conscientiousness': 'M...  
1  {'Openness': 'Medium', 'Conscientiousness': 'H...  
2  {'Openness': 'Medium', 'Conscientiousness': 'M...  
3  {'Openness': 'Medium', 'Conscientiousness': 'M...  
4  {'Openness': 'Medium', 'Conscientiousness': 'M...  

Saving results to topics_sadness_mapped_to_big5_user_keywords.csv...
